In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 999)
df_a = pd.read_csv("ERSA.csv")
df_d = pd.read_csv("ERSD.csv")
df_s = pd.read_csv("ERSS.csv")


In [75]:
from datetime import datetime
df_a["admission_date"] = df_a["ADMISSION_STATUS_DATE"].apply(lambda x: datetime.strptime(x, "%d-%b-%y"))

df_admission = df_a[["EMPLID","admission_date"]]
df_admission.shape

(83694, 2)

In [3]:
def QBXEncode(x):
    l = ['QBX']
    code = {"0" : "A", "1" : "B", "2" : "C","3":"T","4":"U","5":"V","6":"X","7":"Y","8":"Z","9":"K" }
    for i, v in enumerate(list(str(x)),start=1):
        if i % 2 ==0:
            l.append(code[str(v)])
        else:
            l.append(str(v))
    return "".join(l)

def get_clean_data(df):
    dfCBA=df[((df['STUDENT_COLLEGE_CODE']=='CBA') | (df['CLASS_COLLEGE_CODE']=='CBA')) & (df['STUDENT_ACAD_PROG_PRIMARY_CODE']=='UGD')]
    dfCBA = dfCBA.sort_values(by=['EMPLID','CLASS_TERM'])

    df_clean =dfCBA[['CTERM_TERM_LDESC','CLASS_TERM','EMPLID','STUDENT_ACAD_CAREER','STUDENT_ACAD_PROG_PRIM_LDESC','STUDENT_COLLEGE_CODE','STUDENT_DEPT_CODE','STUDENT_ACAD_PLAN','STUDENT_SUBACAD_PLAN',
             'CLASS_SID','CLASS_COLLEGE_CODE','CLASS_DEPT','CLASS_SUBDEPT','CLASS_I','CLASS_DESCR','ONLINE_COURSE','ENRL_UNITS_TAKEN','ENRL_GRADING_BASIS','ENRL_OFFICIAL_GRADE',
                  'CLASS_STDN_MTG_PAT','CLASSM_MEETING_TIME_START','CLASSM_MONDAY','CLASSM_TUESDAY',
                  'CLASSM_WEDNESDAY','CLASSM_THURSDAY','CLASSM_FRIDAY','CLASSM_SATURDAY','CLASSM_SUNDAY','STUDENT_TYPE_FINAL','RES_TUITION_RESIDENCY']]
    
    return df_clean

In [4]:
df_dataset_1 =pd.read_csv('dataset-Encrypted-2012-13.csv')
df_dataset_2 =pd.read_csv('dataset2-Encrypted-2012-13.csv')
print(df_dataset_1.shape)
print(df_dataset_2.shape)

df_dataset = pd.concat([df_dataset_1, df_dataset_2])
print(df_dataset.shape)
df1 = get_clean_data(df_dataset)
print(df1.shape)

(256112, 133)
(181396, 133)
(437508, 133)
(49882, 30)


In [5]:
df_encrypted_1=pd.read_csv('encrypted-2009-2012.csv')
df_encrypted_2=pd.read_csv('encrypted-2009-2012-part2.csv')

print(df_encrypted_1.shape)
print(df_encrypted_2.shape)
df_encrypted = pd.concat([df_encrypted_1, df_encrypted_2])
print(df_encrypted.shape)
df2 = get_clean_data(df_encrypted)
print(df2.shape)

(949153, 127)
(31060, 127)
(980213, 127)
(122476, 30)


In [6]:
df_spring = pd.read_csv('Spring2014dataset.csv', encoding="ISO-8859-1")
df = df_spring[((df_spring['STUDENT_COLLEGE_CODE']=='CBA') | (df_spring['CLASS_COLLEGE_CODE']=='CBA')) & (df_spring['STUDENT_ACAD_PROG_PRIMARY_CODE']=='UGD')]
df3 = df.sort_values(by=['EMPLID','CLASS_TERM'])
df3['EMPLID']= df3['EMPLID'].apply(lambda x: '{0:0>9}'.format(x))
def QBXEncode(x):
    l = ['QBX']
    code = {"0" : "A", "1" : "B", "2" : "C","3":"T","4":"U","5":"V","6":"X","7":"Y","8":"Z","9":"K" }
    for i, v in enumerate(list(str(x)),start=1):
        if i % 2 ==0:
            l.append(code[str(v)])
        else:
            l.append(str(v))
    return "".join(l)

#list
df3['EMPLID']=df3['EMPLID'].apply(QBXEncode)
df3=df3[['CTERM_TERM_LDESC','CLASS_TERM','EMPLID','STUDENT_ACAD_CAREER','STUDENT_ACAD_PROG_PRIM_LDESC','STUDENT_COLLEGE_CODE','STUDENT_DEPT_CODE','STUDENT_ACAD_PLAN','STUDENT_SUBACAD_PLAN',
             'CLASS_SID','CLASS_COLLEGE_CODE','CLASS_DEPT','CLASS_SUBDEPT','CLASS_I','CLASS_DESCR','ONLINE_COURSE','ENRL_UNITS_TAKEN','ENRL_GRADING_BASIS','ENRL_OFFICIAL_GRADE',
                  'CLASS_STDN_MTG_PAT','CLASSM_MEETING_TIME_START','CLASSM_MONDAY','CLASSM_TUESDAY',
                  'CLASSM_WEDNESDAY','CLASSM_THURSDAY','CLASSM_FRIDAY','CLASSM_SATURDAY','CLASSM_SUNDAY','STUDENT_TYPE_FINAL','RES_TUITION_RESIDENCY']]
print(df3.shape)

(15895, 30)


In [7]:
df_student_lvl=pd.read_csv('SECTION_STUDENT_LVL_DATA-2017-2014.csv')
df4 = get_clean_data(df_student_lvl)
print(df4.shape)

(113787, 30)


In [8]:
dataList = [df1,df2, df3,df4]
for data in dataList:
    print(data.shape)
dfCBA = pd.concat(dataList).sort_values(by=['EMPLID','CLASS_TERM'])
print(dfCBA.shape)

(49882, 30)
(122476, 30)
(15895, 30)
(113787, 30)
(302040, 30)


In [164]:
import numpy as np

#select from 11 classes
cbaClassList = ["Business Communications","Business Finance","Business Statistics I","Business Strategy & Policy","International Business","Legal & Regulatry Env Business","Management Information Systems","Marketing","Organizational Behavior","Princip of Mgmt and Operations","Principles of Management"]
cbaClassCode = ['HRM 360', 'MKTG 300', 'MGMT 425', 'I S 310', 'MGMT 300', 'FIN 300', 'BLAW 320', 'I S 300', 'CBA 300', 'I S 301']
quantitativeList = ['FIN 300', 'I S 300', 'I S 301']

dfCBA11 =  dfCBA[dfCBA['CLASS_DESCR'].isin(cbaClassList)]

#select only valid scores: ['A', 'B', 'C', 'D', 'F']
scoreDict = {"A" : 4, "B" : 3, "C" : 2, "D": 1, "F": 0}
scoreList = list(scoreDict.keys())
 
df =  dfCBA11[ dfCBA11['ENRL_OFFICIAL_GRADE'].isin(scoreList)]

majorList = ['Accountancy BS', 'Finance BS', 'Human Resources Management BS', 'International Business BS',  'Management BS',  'Marketing BS',  'Mgmt Info Systems BS', 'Oper Supply Chain Mgmt  BS', 'Operations Management BS']
df_ttest = df[df['STUDENT_ACAD_PLAN'].isin(majorList)]

df_ttest["score"] = df_ttest["ENRL_OFFICIAL_GRADE"].map(scoreDict)
df_ttest["lesson"] = df_ttest["CLASS_I"].apply(lambda x: x[:-3])  


df_ttest['units']= df_ttest['ENRL_UNITS_TAKEN'].apply(lambda x: int(x))
df_gpa = df_ttest.groupby('EMPLID').apply(lambda x: np.average(x["score"], weights = x["units"], axis = 0)).to_frame(name = 'GPA').reset_index()
df_ttest = df_ttest.merge(df_gpa, on = ['EMPLID'], how ='left')
print(df_ttest.shape)

df_quantGPA = df_ttest[['EMPLID', 'score', 'lesson']].loc[df_ttest['lesson'].isin(quantitativeList)]
df_quantGPA1 = df_quantGPA.groupby('EMPLID').agg({'score':'mean'}).rename(columns={'score':'quantGPA'}).reset_index()
df_ttest = df_ttest.merge(df_quantGPA1, on = ['EMPLID'], how ='left') 

(79917, 34)


# Class score description for Management Information Systems

In [11]:
def describe(className):
    df = df_ttest[ df_ttest['CLASS_DESCR']== className  ]
    return df.groupby(["STUDENT_ACAD_PLAN"])[["score"]].describe()

In [12]:
d = describe("Management Information Systems")
d

score                                          \
                                count      mean       std  min  25%  50%  75%   
STUDENT_ACAD_PLAN                                                               
Accountancy BS                 1781.0  2.824818  0.848443  0.0  2.0  3.0  3.0   
Finance BS                     1265.0  2.732016  0.848562  0.0  2.0  3.0  3.0   
Human Resources Management BS   361.0  2.645429  0.810718  0.0  2.0  3.0  3.0   
International Business BS       449.0  2.652561  0.834255  0.0  2.0  3.0  3.0   
Management BS                  1434.0  2.638773  0.847345  0.0  2.0  3.0  3.0   
Marketing BS                   1401.0  2.659529  0.815298  0.0  2.0  3.0  3.0   
Mgmt Info Systems BS            433.0  2.854503  0.852330  0.0  2.0  3.0  3.0   
Oper Supply Chain Mgmt  BS       85.0  2.858824  0.847306  0.0  2.0  3.0  3.0   
Operations Management BS         11.0  3.090909  0.831209  2.0  2.5  3.0  4.0   

                                    
                               max  
STUDENT_ACAD_PLAN                   
Accountancy BS                 4.0  
Finance BS                     4.0  
Human Resources Management BS  4.0  
International Business BS      4.0  
Management BS                  4.0  
Marketing BS                   4.0  
Mgmt Info Systems BS           4.0  
Oper Supply Chain Mgmt  BS     4.0  
Operations Management BS       4.0

# Hypothesis1 : Higher score for major student

In [13]:
import scipy 
import scipy.stats
import numpy as np

majorList.sort()

def run_ttest_on_class_major(classCode, major):
    df = df_ttest[ df_ttest['lesson']== classCode  ]
    group1 = df[ df['STUDENT_ACAD_PLAN'] == major ]
    group2 = df[ df['STUDENT_ACAD_PLAN'] != major ]
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())

majorClassList = [('Human Resources Management BS', 'HRM 360'), ('Management BS', 'MGMT 300'), ('Marketing BS', 'MKTG 300'), \
                  ('International Business BS','CBA 300'), ('Mgmt Info Systems BS', 'I S 300'), ('Mgmt Info Systems BS', 'I S 301'), ('Finance BS', 'FIN 300')]
     
t_test_results = {}
scores = {}
for  majorName, className in majorClassList:
    t_test_results[className + " : " + majorName], scores[className + " : " + majorName ] = run_ttest_on_class_major(className, majorName)

#print(t_test_results)
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['majorScore'] = scores_df[0].to_numpy()
results_df['nonMajorScore'] = scores_df[1].to_numpy()
results_df

,statistic,pvalue,majorScore,nonMajorScore
HRM 360 : Human Resources Management BS,-0.455351,6.488684e-01,2.557292,2.580923
MGMT 300 : Management BS,-4.078622,4.571056e-05,2.564351,2.660422
MKTG 300 : Marketing BS,-0.432440,6.654329e-01,2.642953,2.653528
CBA 300 : International Business BS,-0.701952,4.827293e-01,2.444444,2.473967
I S 300 : Mgmt Info Systems BS,3.364837,7.698818e-04,2.854503,2.714012
I S 301 : Mgmt Info Systems BS,-1.074764,2.825129e-01,2.623404,2.659043
FIN 300 : Finance BS,5.078887,3.880284e-07,2.851661,2.693636


# Hypothesis 2: Different class time has score difference

In [14]:
df_ttest["hour"] = df_ttest["CLASSM_MEETING_TIME_START"].str.split(":").str[0]
df_ttest['hour'] = df_ttest['hour'].replace(np.nan, 0)

df_ttest["morning"] = df_ttest["hour"].apply(lambda x: 1 if int(x) < 12 else  0)
df_ttest["afternoon"] = df_ttest["hour"].apply(lambda x: 1 if int(x) >= 12 and int(x) < 18 else  0)
df_ttest["night"] = df_ttest["hour"].apply(lambda x: 1 if int(x) >= 18  else  0)

def run_ttest_on_day_time(dayTime):
    group1 = df_ttest[ df_ttest[dayTime] == 1 ]
    group2 = df_ttest[ df_ttest[dayTime] == 0 ]
    
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())
 
t_test_results = {}
scores = {}
for dayTime in ["morning", "afternoon", "night"]:
    t_test_results[dayTime], scores[dayTime] = run_ttest_on_day_time(dayTime)

results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['inPeriodScore'] = scores_df[0].to_numpy()
results_df['outPeriodScore'] = scores_df[1].to_numpy()
results_df

,statistic,pvalue,inPeriodScore,outPeriodScore
morning,-0.07468,0.940470,2.668255,2.668778
afternoon,1.12537,0.260436,2.672453,2.665179
night,-1.26825,0.204712,2.660967,2.670808


# Hypothesis 3: Students take 2 or more quantitative classes has lower score

In [221]:
quantitativeList = ['FIN 300', 'I S 300', 'I S 310']
df_ttest["quantitative"] = df_ttest['lesson'].apply(lambda x: 1 if x in quantitativeList else  0)
#df_ttest.head(10)

df2 = df_ttest.groupby(['EMPLID', 'CTERM_TERM_LDESC'])['quantitative'].sum().reset_index().rename(columns={'quantitative':'sum_quantitative'})
df_hyp3 = df_ttest.merge(df2, on = ['EMPLID','CTERM_TERM_LDESC'], how ='left')

#print(df_ttest.shape)
print(df_quant.shape)

def run_ttest_on_quant():
    group1 = df_hyp3[ df_hyp3["sum_quantitative"] >= 2 ]
    group2 = df_hyp3[ df_hyp3["sum_quantitative"] <  2 ]
    
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())
 
t_test_results = {}
scores = {}
t_test_results["Hypothesis3"], scores["Hypothesis3"] = run_ttest_on_quant()
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')
results_df['MoreQuantClassScore'] = scores_df[0].to_numpy()
results_df['LessQuantClassScore'] = scores_df[1].to_numpy()
results_df

(79917, 37)


,statistic,pvalue,MoreQuantClassScore,LessQuantClassScore
Hypothesis3,2.144264,0.032015,2.684215,2.665548


# Hypothesis 4: Which Major has higher quantitive class GPA?

In [207]:
df_hyp4 = df_ttest[['EMPLID', 'quantGPA', 'STUDENT_ACAD_PLAN']].loc[df_ttest["quantGPA"].notnull()]
df_hyp4 = df_hyp4.drop_duplicates() 
print(df_hyp4.shape)

df_hyp4["major_mis"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Mgmt Info Systems BS' else  0) 
df_hyp4["major_fin"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Finance BS' else  0) 
df_hyp4["major_acc"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Accountancy BS' else  0) 
df_hyp4["major_mrk"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Marketing BS' else  0) 
df_hyp4["major_mgmt"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Management BS' else  0) 
df_hyp4["major_hrm"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Human Resources Management BS' else  0) 
df_hyp4["major_oscm"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Oper Supply Chain Mgmt  BS' or x == 'Operations Management BS' else  0) 
df_hyp4["major_ib"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'International Business BS' else  0) 

def run_ttest_on_major(major):
    group1 = df_hyp4[ df_hyp4[major] == 1 ]
    group2 = df_hyp4[ df_hyp4[major] == 0 ]
    
    return scipy.stats.ttest_ind(group1['quantGPA'], group2['quantGPA']), (group1["quantGPA"].mean(), group2["quantGPA"].mean())
 
t_test_results = {}
scores = {}
for major in ["major_mis", "major_fin", "major_acc", "major_mrk", "major_mgmt", "major_hrm", "major_oscm", "major_ib"]:
    t_test_results[major], scores[major] = run_ttest_on_major(major)

results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['majorQuantGPA'] = scores_df[0].to_numpy()
results_df['otherMajorQuantGPA'] = scores_df[1].to_numpy()
results_df


(11499, 3)


,statistic,pvalue,majorQuantGPA,otherMajorQuantGPA
major_mis,0.625923,5.313780e-01,2.717515,2.699481
major_fin,3.236484,1.213545e-03,2.747282,2.690362
major_acc,10.989158,5.935088e-28,2.830146,2.658651
major_mrk,-5.774349,7.925122e-09,2.619795,2.719234
major_mgmt,-6.538710,6.468009e-11,2.613131,2.722667
major_hrm,-2.543091,1.100065e-02,2.626387,2.704489
major_oscm,-0.555383,5.786435e-01,2.670764,2.701000
major_ib,-2.399908,1.641502e-02,2.637963,2.704708


# Hypothesis 5: Students in junior year taking quantitative class have lower score

In [126]:
from datetime import timedelta
    
df_5 = df_quant.merge(df_admission, on = ['EMPLID'], how ='left')

df_5["course_date"] = df_5["CTERM_TERM_LDESC"].apply(lambda x: datetime.strptime(x.replace("Fall","09 01").replace("Spring","01 01"), "%m %d %Y"))
df_5["start_time"]  = df_5.course_date - df_5.admission_date
df_5["junior_year"] = df_5["start_time"].apply(lambda x: 1 if x >  timedelta(days=365*2) else 0)

df_5 = df_5.loc[df_5["admission_date"].notnull()]
df_5 = df_5.loc[df_5["quantitative"] == 1]

print(df_5.shape)
#print(df_5[["CTERM_TERM_LDESC", "course_date", "admission_date", "start_time", "junior_year", "quantitative"]])

def run_ttest_on_h5(df, check_gpa = False):
    group1 = df[ df["junior_year"] == 1 ]
    group2 = df[ df["junior_year"] == 0 ]
    
    if check_gpa:
        return scipy.stats.ttest_ind(group1['GPA'], group2['GPA']), (group1["GPA"].mean(), group2["GPA"].mean())
    else:
        return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())
    
    
t_test_results = {}
scores = {}
t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5)
results_gpa = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')
results_gpa['Junior'] = scores_df[0].to_numpy()
results_gpa['Senior'] = scores_df[1].to_numpy()

def run_hypothesis5(lesson):
    t_test_results = {}
    scores = {}
    if lesson == "GPA":
        t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5, True)
    else:
        t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5.loc[df_5["lesson"] == lesson])
 
    results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
    scores_df = pd.DataFrame.from_dict(scores, orient='Index')
    results_df['Junior'] = scores_df[0].to_numpy()
    results_df['Senior'] = scores_df[1].to_numpy()
    results_df['Class Name'] = lesson
    return results_df
    
df_hp5_gpa = run_hypothesis5("GPA")
df_hp5_fin300 = run_hypothesis5("FIN 300")
df_hp5_is300 = run_hypothesis5("I S 300")
df_hp5_is301 = run_hypothesis5("I S 301")

df_dataset = pd.concat([df_hp5_gpa, df_hp5_fin300, df_hp5_is300, df_hp5_is301])
df_dataset

(5359, 40)


,statistic,pvalue,Junior,Senior,Class Name
Hypothesis5,-7.359891,2.119330e-13,2.698736,2.966794,GPA
Hypothesis5,-3.696094,2.253922e-04,2.665299,3.016129,FIN 300
Hypothesis5,0.157474,8.748906e-01,2.754975,2.738462,I S 300
Hypothesis5,-2.968016,3.035694e-03,2.689733,2.892473,I S 301


# Unsupvised Learning: K means Clusting on GPA 

In [219]:
from sklearn.cluster import KMeans

df_cluster = df_ttest[["EMPLID", "quantGPA"]][df_ttest["quantGPA"].notnull()].drop_duplicates().sort_values('quantGPA')

# Cluster students based on their quantitive class GPAs into 5 different groups 
X = np.asarray(df_cluster["quantGPA"])
kmeans = KMeans(n_clusters=5).fit(X.reshape(-1,1))  

# Save the labels
df_cluster.loc[:,'GPA_cluster'] = kmeans.labels_
 
# For each student, calculate the maximum amount of quantitive classes in a semaster
df_temp = df_hyp3[["EMPLID", "CTERM_TERM_LDESC", "sum_quantitative"]].drop_duplicates()
df_temp1 = df_temp.groupby('EMPLID').agg({'sum_quantitative':'max'}).rename(columns={'sum_quantitative':'maxQuantClass'}).reset_index()
print(df_cluster[["quantGPA"]].groupby(df_cluster["GPA_cluster"]).agg(['mean', 'count']))
#print(df_cluster.shape)

df_cluster1 = df_cluster.merge(df_temp1, on = ['EMPLID'], how ='left')

#print(df_cluster1.shape)

# For each cluster, count number of students with 
print(df_cluster1[["quantGPA","maxQuantClass"]].groupby(df_ttest["GPA_cluster"]).agg(['mean', 'count']))
#print(df_cluster[["GPA"]].groupby(df_ttest["GPA_cluster"]).agg(['mean', 'count']))

             quantGPA      
                 mean count
GPA_cluster                
0            1.917728  2168
1            2.506177  2466
2            3.624592  2286
3            3.000132  2650
4            0.845906   399
             quantGPA       maxQuantClass      
                 mean count          mean count
GPA_cluster                                    
0            2.767828  2884      1.481969  2884
1            2.650622  1501      1.419054  1501
2            2.671439  2497      1.484181  2497
3            2.720368   689      1.496372   689
4            2.671240  2398      1.492494  2398


In [172]:
grouped = df_cluster['score'].groupby(df_cluster['EMPLID']).mean()
grouped.index.name = 'index'
 
df2 = grouped.to_frame() #pd.DataFrame({'EMPLID':grouped.index, 'mean':grouped.values})
df2.rename(columns = {'score':'avgGPA'}, inplace = True) 
df2['EMPLID'] = list(grouped.index)

df_cluster1 = df_cluster.merge(df2, on='EMPLID', how='left')
df_cluster1.shape

(79917, 39)